In [1]:
# !pip install --upgrade transformers==4.40.0 diffusers==0.27.2 ftfy==6.2.0 accelerate datasets bitsandbytes

In [5]:
!pip install --upgrade transformers==4.40.0 diffusers==0.27.2 ftfy==6.2.0 accelerate datasets bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 57.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 9.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 32.2 MB/s eta 0:00:0000:01:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.0
    Uninstalling tokenizers-0.20.0:
      Successfully uninstalled tokenizers-0.20.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accel

In [2]:
!pip install nest-asyncio pyngrok uvicorn

In [3]:
!ngrok config add-authtoken 2o5bryAuCTdJ7n8kNjkLkDVCmI0_2jzXf8asm6z2DbdNw9V31

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [4]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

# middlewares
app.add_middleware(
    CORSMiddleware, # https://fastapi.tiangolo.com/tutorial/cors/
    allow_origins=['*'], # wildcard to allow all, more here - https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Origin
    allow_credentials=True, # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Credentials
    allow_methods=['*'], # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Methods
    allow_headers=['*'], # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Headers
)

# @app.get('/')
# async def root():
#     return {'hello': 'world'}

In [6]:
from model import StableDiffusionModel, ImageCaptioningPipeline

In [7]:
from fastapi import FastAPI, Form, HTTPException
from fastapi.responses import FileResponse, JSONResponse
import torch
from diffusers import StableDiffusionPipeline
import os
from PIL import Image
import uuid

# Initialize FastAPI app
app = FastAPI()

# # Load the Stable Diffusion model
# model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda" if torch.cuda.is_available() else "cpu"

# try:
#     pipe = StableDiffusionPipeline.from_pretrained(model_id)
#     pipe = pipe.to(device)
# except Exception as e:
#     raise RuntimeError(f"Failed to load model: {str(e)}")

# Endpoint to generate image based on user prompt
@app.post("/generate")
async def generate_image(prompt: str = Form(...)):
    # Validate prompt
    if not prompt or len(prompt.strip()) == 0:
        raise HTTPException(status_code=400, detail="Prompt cannot be empty")

    # Generate image from prompt
    try:
        with torch.cuda.amp.autocast() if device == "cuda" else torch.no_grad():
            model = StableDiffusionModel()
            negative_prompts = 'bad anatomy, bad proportions, blurry, cloned face, cropped, deformed, dehydrated, disfigured, duplicate, error, extra arms, extra fingers, extra legs, extra limbs, fused fingers, gross proportions, jpeg artifacts, long neck, low quality, lowres, malformed limbs, missing arms, missing legs, morbid, mutated hands, mutation, mutilated, out of frame, poorly drawn face, poorly drawn hands, signature, text, too many fingers, ugly, username, watermark, worst quality'
            text_embeddings = model.prompt_to_emb(prompt, negative_prompts)
            latents = model.emb_to_latents(text_embeddings)
            image = model.latents_to_pil(latents)
            image = image[0]
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Failed to generate image: {str(e)}")
        
    # Save the generated image to a file with a unique name
    output_filename = f"output_{uuid.uuid4().hex}.png"
    output_path = os.path.join("./generated_images", output_filename)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    image.save(output_path)

    # Return the image file as a response
    return FileResponse(output_path, media_type="image/png", filename=output_filename)

# # Endpoint to caption an uploaded image
# @app.post("/caption")
# async def caption_image(file: UploadFile = File(...)):
#     # Validate file type
#     if file.content_type not in ["image/jpeg", "image/png"]:
#         raise HTTPException(status_code=400, detail="Invalid file type. Please upload a JPEG or PNG image.")

#     # Read the uploaded image
#     try:
#         image = Image.open(BytesIO(await file.read()))
#     except Exception as e:
#         raise HTTPException(status_code=400, detail=f"Failed to read image: {str(e)}")

#     caption_model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large", torch_dtype=torch_dtype, trust_remote_code=True).to(device)
#     processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True)
#     # Prepare inputs for the caption model
#     prompt = "<OD>"
#     inputs = caption_processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

#     # Generate caption
#     try:
#         generated_ids = caption_model.generate(
#             input_ids=inputs["input_ids"],
#             pixel_values=inputs["pixel_values"],
#             max_new_tokens=1024,
#             num_beams=3,
#             do_sample=False
#         )
#         generated_text = caption_processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
#         parsed_answer = caption_processor.post_process_generation(generated_text, task="<OD>", image_size=(image.width, image.height))
#     except Exception as e:
#         raise HTTPException(status_code=500, detail=f"Failed to generate caption: {str(e)}")

#     # Return the caption as a response
#     return JSONResponse(content={"caption": parsed_answer})

# Health check endpoint to verify that the service is running
@app.get("/health")
async def health_check():
    return JSONResponse(content={"status": "healthy"})

# # For local testing only
# if __name__ == "__main__":
#     import uvicorn
#     uvicorn.run(app, host="0.0.0.0", port=8000)

# Running on Kaggle:
# Make sure to expose the port to the internet using Kaggle's network setting,
# and replace the host and port with appropriate values for Kaggle.

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn


# specify a port
port = 8000
ngrok_tunnel = ngrok.connect(port)

# where we can visit our fastAPI app
print('Public URL:', ngrok_tunnel.public_url)


nest_asyncio.apply()

# finally run the app
uvicorn.run(app, port=port)

INFO:     Started server process [30]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://9a83-35-204-223-60.ngrok-free.app


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

scheduler/scheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

INFO:     99.216.134.90:0 - "POST /generate HTTP/1.1" 500 Internal Server Error


In [ ]:
# prompt = ["a photograph of an astronaut riding a horse"]

# text_embeddings = prompt_to_emb(prompt, negative_prompts)
# latents = emb_to_latents(text_embeddings)
# image = latents_to_pil(latents)
# image[0]

In [ ]:
import requests

import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM 


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large", torch_dtype=torch_dtype, trust_remote_code=True).to(device)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True)

prompt = "<OD>"

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=1024,
    num_beams=3,
    do_sample=False
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

parsed_answer = processor.post_process_generation(generated_text, task="<OD>", image_size=(image.width, image.height))

print(parsed_answer)